In [ ]:
"""
01, cap at .99, 0.387178, 0.384541
"""

In [ ]:
"""
categorical only + various estimator
numerical only + various estimator
variable selection (xgb, RF) + various estimator
numerical as categorical
second order features
standardize kfold transformer
numerical as categorical + 2nd order + linear
"""

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import xgboost as xgb
import vw_utils as vw
import gini
%matplotlib inline

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

cap = train.Hazard.quantile(0.99)
print(cap)

train['Hazard_cap'] = np.minimum(train.Hazard, cap)

data = pd.concat([train, test], axis=0).reset_index()
train_set = np.hstack([np.ones(train.shape[0]), np.zeros(test.shape[0])])

19.0


## --------------------- Feature Start -----------------------

In [3]:
# start
categorical = ['T1_V' + str(i) for i in list(range(4, 10)) + [11, 12, 15, 16, 17]] + \
                ['T2_V' + str(i) for i in [3, 5, 11, 12, 13]]

numerical = set(train.columns).difference(categorical + ['Hazard', 'Hazard_cap', 'Id'])
numerical = list(numerical)

In [4]:
# raw feature
from sklearn.preprocessing import StandardScaler
import re

SS = StandardScaler()
feat_raw = data[numerical].applymap(float)
feat_raw[numerical] = SS.fit_transform(feat_raw[numerical])


toDrop = re.sub('[ ]+', '_', data[categorical].ix[0, :].to_string()).split('\n')
feat_raw_cat = pd.get_dummies(data[categorical]).drop(toDrop, axis=1)

feat_raw = pd.concat([feat_raw, feat_raw_cat], axis=1)
feat_raw.shape

(101999, 95)

In [5]:
%%time
# count features
from wrappers import generatePrimes, calPowerCount
from sklearn.preprocessing import StandardScaler

feat_count = pd.concat([calPowerCount(data[categorical], 1), 
                        calPowerCount(data[categorical], 2),
                        calPowerCount(data[categorical], 3)], axis=1)

cols = feat_count.columns
feat_count = pd.DataFrame(StandardScaler().fit_transform(feat_count.values), columns=cols)

CPU times: user 15.8 s, sys: 12.4 s, total: 28.1 s
Wall time: 28.3 s


/home/albertium/anaconda3/lib/python3.4/site-packages/sklearn/utils/validation.py:498: UserWarning: StandardScaler assumes floating point values as input, got int64
  "got %s" % (estimator, X.dtype))


In [37]:
# assemble features

tmp = pd.concat([feat_raw, feat_count], axis=1)
train_c = tmp[train_set==1]
test_c = tmp[train_set==0]

print(train_c.shape)
print(test_c.shape)

(50999, 1063)
(51000, 1063)


## --------------------- Modeling -----------------------

In [38]:
%%time
from sklearn.linear_model import ElasticNet
from sklearn.cross_validation import KFold

model = ElasticNet(alpha=1E-3, l1_ratio=0.5, max_iter=2000)
scores = []
time = 1
for idx_train, idx_test in KFold(train.shape[0], 10):
    print(time)
    time += 1    
    model.fit(train_c.ix[idx_train, :], train.Hazard[idx_train])
    pred = model.predict(train_c.ix[idx_test, :])
    scores.append(gini.normalized_gini(train.Hazard[idx_test], pred))

print(np.mean(scores))

1
2
3
4
5
6
7
8
9
10
0.331175260261
CPU times: user 33min 7s, sys: 0 ns, total: 33min 7s
Wall time: 33min 8s


/home/albertium/anaconda3/lib/python3.4/site-packages/sklearn/linear_model/coordinate_descent.py:444: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


In [141]:
import subprocess
import sys

command = \
"""
--passes 5 -k -c -f tmp.model
--readable_model tmp.model.r
--loss_function squared --l1 1E-5 --l2 1E-5
"""

proc = subprocess.Popen(['vw', command], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
vwTrain = py2vw(train_c, {'A': train_c.columns}, 'Hazard')
proc.stdin.write(vwTrain)
proc.stdin.write(vwTrain)
proc.stdin.flush()
out, err = proc.communicate()
print(out)
print(err.decode('UTF-8'))

KeyError: "['Hazard'] not in index"

In [120]:
%%time
def py2vw(data, namespaces, label=None):
    data = data.copy()
    ordering = [label] if label else []
    
    for ns, cols in namespaces.items():
        ordering.append(ns)
        data[ns] = '|' + ns
        
        for col in cols:
            ordering.append(col)            
            if data[col].dtype == 'object':
                data[col] = col + '_'+ data[col]
            else:
                data[col] = col + ':' + data[col].map(str)
                
    return data[ordering].to_csv(index=False, header=False, sep=' ').encode('UTF-8')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10 µs


In [44]:
pd.DataFrame({'Id': test.Id, 'Hazard': yhat}).reindex_axis(['Id', 'Hazard'], 1).to_csv('../output/ensemble_07.csv', index=0)